In [2]:
import requests
import csv
import json
import pandas as pd

# Function to remove Base64 fields from response data
def remove_base64_fields(data):
    if isinstance(data, dict):
        return {key: remove_base64_fields(value) for key, value in data.items() if key.lower() != 'base64'}
    elif isinstance(data, list):
        return [remove_base64_fields(item) for item in data]
    else:
        return data

# Function to process a single auction ID to fetch, clean, and extract relevant data
def process_auction_id(auction_id, headers):
    url = "https://eaukcija.sud.rs/WebApi.Proxy/api/EAukcija/GetMovablePropertyDetails"
    data_payload = {"AuctionId": auction_id, "Role": None}
    response = requests.post(url, headers=headers, json=data_payload)
    
    if response.status_code == 200:
        cleaned_data = remove_base64_fields(response.json())
        if 'Data' in cleaned_data:
            data_section = cleaned_data['Data']
            category_data = data_section.get("Category", {})
            place_data = data_section.get("Place", {})
            return {
                "ExecutorName": data_section.get("ExecutorName", ""),
                "EstimatedPrice": data_section.get("EstimatedPrice", ""),
                "AuctionNumber": data_section.get("AuctionNumber", ""),
                "Id": data_section.get("Id", ""),
                "Description": data_section.get("Description", ""),
                "NameCategory": category_data.get("Name", ""),
                "Code": place_data.get("Code", ""),
                "Name": place_data.get("Name", ""),
                "ZipCode": place_data.get("ZipCode", ""),
                "Municipality": place_data.get("Municipality", ""),
                "Cadastral": place_data.get("Cadastral", ""),
                "ParcelNumber": place_data.get("ParcelNumber", "")
            }
    else:
        print(f"Failed to process AuctionId: {auction_id}, Status Code: {response.status_code}")
        return {}

# Initial setup: URL, headers, and first request to determine TotalCount
url = "https://eaukcija.sud.rs/WebApi.Proxy/api/EAukcija/GetAuctionsByCategoryId"
headers = {
    "accept": "application/json",
    "accept-language": "en,en-US;q=0.9,sr;q=0.8,bs;q=0.7,hr;q=0.6,fi;q=0.5",
    "content-type": "application/json;charset=UTF-8"
    #"authorization": "bearer YOUR_ACCESS_TOKEN",  # Replace with your actual token
}
initial_data = {"CategoryId": "2", "ItemCount": 1, "PageCount": "1"}
response = requests.post(url, json=initial_data, headers=headers)
total_count = response.json().get('Data', {}).get('TotalCount', 0)

# Second request: fetch all auctions
full_data = {"CategoryId": "2", "ItemCount": total_count, "PageCount": "1"}
response_full = requests.post(url, json=full_data, headers=headers)
auctions_full = response_full.json().get('Data', {}).get('Auctions', [])

# Extract and process auction details for each auction
results = []
for auction in auctions_full:
    auction_id = auction['Id']
    auction_details = process_auction_id(auction_id, headers)
    if auction_details:
        auction.update(auction_details)
        results.append(auction)

# Combine the results into a DataFrame
fieldnames = [
    'Id', 'AuctionNumber', 'StartDate', 'EndDate', 'EstimatedPrice', 'StartingPrice',
    'IsFirstSale', 'PropertyType', 'NameCategory', 'ShortDescription', 'Description',
    'Code', 'Name', 'ZipCode', 'Municipality', 'Cadastral', 'ParcelNumber',
    'Status', 'StatusTranslation', 'NumberOfVerifiedUsers', 'ExecutorName'
]
df = pd.DataFrame(results, columns=fieldnames)

# Calculate additional columns
df['StartingPrice/EstimatedPrice'] = df['StartingPrice'] / df['EstimatedPrice']
df['EstimatedPrice-StartingPrice'] = df['EstimatedPrice'] - df['StartingPrice']

# Reorder columns
new_order = [
    'Id', 'AuctionNumber', 'StartDate', 'EndDate', 'EstimatedPrice', 'StartingPrice',
    'StartingPrice/EstimatedPrice', 'EstimatedPrice-StartingPrice', 'IsFirstSale',
    'PropertyType', 'NameCategory', 'ShortDescription', 'Description', 'Code', 'Name',
    'ZipCode', 'Municipality', 'Status', 'StatusTranslation', 'NumberOfVerifiedUsers',
    'ExecutorName'
]
df = df[new_order]

# Save the final DataFrame to a CSV file
output_file_path = 'aukcije_pokretnosti.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"Final CSV file created: {output_file_path}")


Final CSV file created: aukcije_pokretnosti.csv


## SEGMENTI

In [1]:
import requests
import json

# Initial setup: URL and headers
url = "https://eaukcija.sud.rs/WebApi.Proxy/api/EAukcija/GetAuctionsByCategoryId"
headers = {
    "accept": "application/json",
    "accept-language": "en,en-US;q=0.9,sr;q=0.8,bs;q=0.7,hr;q=0.6,fi;q=0.5",
    "content-type": "application/json;charset=UTF-8",
}

# First request: determine TotalCount
initial_data = {
    "CategoryId": "2",
    "ItemCount": 1,  # Minimal item count to reduce initial load
    "PageCount": "1",
}
response = requests.post(url, json=initial_data, headers=headers)
total_count = response.json().get('Data', {}).get('TotalCount', 0)

# Second request: use TotalCount as ItemCount to fetch all auctions
full_data = {
    "CategoryId": "2",
    "ItemCount": total_count,  # Set ItemCount to TotalCount
    "PageCount": "1",
}
response_full = requests.post(url, json=full_data, headers=headers)
auctions_full = response_full.json().get('Data', {}).get('Auctions', [])

# Remove the 'Thumbnail' field from each auction
for auction in auctions_full:
    if 'Thumbnail' in auction:
        del auction['Thumbnail']

# Save the output as a JSON file named 'aukcije_nepokretnosti.json'
json_file_path = 'aukcije_pokretnosti.json'  # Adjust the path as needed
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    # Save the modified auctions data without the Thumbnail field
    json.dump({'TotalCount': total_count, 'Auctions': auctions_full}, json_file, ensure_ascii=False, indent=4)

print(f"Output saved as {json_file_path}")


Output saved as aukcije_pokretnosti.json


konvert to csv

In [2]:
import json
import csv

# Read the JSON file
json_file_path = 'aukcije_pokretnosti.json'
with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
    data = json.load(json_file)

# Extract auctions data, ensure 'Thumbnail' field is excluded
auctions = data['Auctions']
for auction in auctions:
    if 'Thumbnail' in auction:
        del auction['Thumbnail']

# Manually specify the fieldnames to include all desired fields
fieldnames = [
    'Id', 'AuctionNumber', 'StartDate', 'EndDate', 'MaxOfferedPrice', 'CurrentPrice',
    'StartingPrice', 'ShortDescription', 'Status', 'StatusTranslation', 'NumberOfVerifiedUsers',
    'IsFirstSale', 'PropertyType'  # Add any other fields you need
]

# Write the data to a CSV file
csv_file_path = 'aukcije_pokretnosti.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for auction in auctions:
        # Fill missing fields with a default value (e.g., None or '')
        row = {field: auction.get(field, None) for field in fieldnames}
        writer.writerow(row)

print(f"CSV file has been created: {csv_file_path}")


CSV file has been created: aukcije_pokretnosti.csv


get detalje za pokretnosti

In [ ]:
import csv
import requests
import json

def remove_base64_fields(data):
    """Recursively remove Base64 fields from response data."""
    if isinstance(data, dict):
        return {key: remove_base64_fields(value) for key, value in data.items() if key.lower() != 'base64'}
    elif isinstance(data, list):
        return [remove_base64_fields(item) for item in data]
    else:
        return data

def process_auction_id(auction_id, headers):
    """Process a single auction ID to fetch, clean, and extract relevant data."""
    url = "https://eaukcija.sud.rs/WebApi.Proxy/api/EAukcija/GetMovablePropertyDetails"
    data_payload = {
        "AuctionId": auction_id,
        "Role": None,
    }
    response = requests.post(url, headers=headers, json=data_payload)
    
    if response.status_code == 200:
        cleaned_data = remove_base64_fields(response.json())
        
        if 'Data' in cleaned_data:
            data_section = cleaned_data['Data']
            executor_name = data_section.get("ExecutorName", "")
            estimated_price = data_section.get("EstimatedPrice", "")
            auction_number = data_section.get("AuctionNumber", "")
            description = data_section.get("Description", "")
            response_auction_id = data_section.get("Id", "")
            category_data = data_section.get("Category", {})
            category_section = {
                "NameCategory": category_data.get("Name", "")
            }
            place_data = data_section.get("Place", {})
            place_section = {
                "Code": place_data.get("Code", ""),
                "Name": place_data.get("Name", ""),
                "ZipCode": place_data.get("ZipCode", ""),
                "Municipality": place_data.get("Municipality", ""),
                "Cadastral": place_data.get("Cadastral", ""),
                "ParcelNumber": place_data.get("ParcelNumber", "")
            }
            return {
                "ExecutorName": executor_name,
                "EstimatedPrice": estimated_price,
                "AuctionNumber": auction_number,
                "Id": response_auction_id,  # Use a different variable name if necessary
                "Description": description,
                **place_section,  # Merge dictionaries for flat structure
                **category_section
            }
    else:
        print(f"Failed to process AuctionId: {auction_id}, Status Code: {response.status_code}")
        return {}

# File paths
csv_file_path = 'aukcije_pokretnosti.csv'
json_file_path = 'details_pokretnosti_data.json'
output_csv_file_path = 'details_pokretnosti_data.csv'

results = []
headers = {
    "accept": "application/json",
    "accept-language": "en,en-US;q=0.9,sr;q=0.8,bs;q=0.7,hr;q=0.6,fi;q=0.5",
    "content-type": "application/json;charset=UTF-8",
    # Replace with your actual token
    "authorization": "bearer YOUR_ACCESS_TOKEN",
}

with open(csv_file_path, newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        auction_id = row['Id']
        data = process_auction_id(auction_id, headers)
        if data:  # Only add if data is successfully returned
            results.append(data)
        else:
            print(f"No data for AuctionId: {auction_id}")

# Save results to a JSON file
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)

# Save results to a CSV file
with open(output_csv_file_path, mode='w', newline='', encoding='utf-8-sig') as file:
    fieldnames = ['NameCategory', 'ExecutorName', 'EstimatedPrice', 'AuctionNumber', 'Id', 'Description', 'Code', 'Name', 'ZipCode', 'Municipality', 'Cadastral', 'ParcelNumber']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"All data saved to {json_file_path} and {output_csv_file_path}")


join

In [ ]:
import pandas as pd

# Load the two CSV files with utf-8-sig encoding to ensure proper handling of special characters
aukcije_nepokretnosti = pd.read_csv('aukcije_pokretnosti.csv', encoding='utf-8-sig')
details_data = pd.read_csv('details_pokretnosti_data.csv', encoding='utf-8-sig')

# Join the two CSV files on the 'Id' column
merged_data = pd.merge(aukcije_nepokretnosti, details_data, on='Id', how='left')

# Save the merged data to a new CSV file, using utf-8-sig encoding
output_file_path = 'sa_detaljima_aukcije_pokretnosti.csv'
merged_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(output_file_path)


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('sa_detaljima_aukcije_pokretnosti.csv', encoding='utf-8-sig')

# Calculate new columns
df['StartingPrice/EstimatedPrice'] = df['StartingPrice'] / df['EstimatedPrice']
df['EstimatedPrice-StartingPrice'] = df['EstimatedPrice'] - df['StartingPrice']

# Define the new order of the columns
new_order = [
    'Id', 'AuctionNumber_x', 'StartDate', 'EndDate', 'EstimatedPrice', 'StartingPrice',
    'StartingPrice/EstimatedPrice', 'EstimatedPrice-StartingPrice', 'IsFirstSale',
    'PropertyType', 'NameCategory', 'ShortDescription', 'Description', 'Code', 'Name',
    'ZipCode', 'Municipality', 'Status', 'StatusTranslation', 'NumberOfVerifiedUsers',
    'ExecutorName'
]

# Reorder the dataframe according to the new_order list
# Make sure all columns in new_order exist in your dataframe. If not, adjust the list accordingly.
df = df[new_order]

# Save the modified dataframe back to CSV
df.to_csv('sa_detaljima_aukcije_pokretnosti.csv', index=False, encoding='utf-8-sig')

print(output_file_path)
